In [1]:
import pandas as pd
import numpy as np

In [25]:
msa_list = pd.read_csv("data/wikipedia_msa.csv", header=0, delimiter=",", encoding="utf-8")
zip_msa = pd.read_csv("data/zip07_cbsa06.txt", header=0, delimiter=",", encoding="latin1", low_memory=False)
#print(zip_msa[zip_msa['CBSA LSAD'] == 'Metropolitan Statistical Area']['CBSA TITLE'].sort_values().unique())

In [26]:
print(zip_msa.columns)
print(zip_msa.head(10))

Index(['ZIP5', 'ZIP4', 'ZIP9', 'STATE CODE', 'STATE', 'COUNTY CODE',
       'COUNTY NAME', 'CBSA CODE', 'CBSA TITLE', 'CBSA LSAD',
       'METRO DIVISION CODE', 'METRO DIVISION TITLE', 'METRO DIVISION LSAD',
       'CSA CODE', 'CSA TITLE', 'CSA LSAD'],
      dtype='object')
   ZIP5  ZIP4   ZIP9  STATE CODE STATE  COUNTY CODE     COUNTY NAME  \
0  4841   NaN  04841          23    ME           13     Knox County   
1  4843   NaN  04843          23    ME           13     Knox County   
2  4846   NaN  04846          23    ME           13     Knox County   
3  4847   NaN  04847          23    ME           13     Knox County   
4  4848   NaN  04848          23    ME           27    Waldo County   
5  4849   NaN  04849          23    ME           27    Waldo County   
6  4850   NaN  04850          23    ME           27    Waldo County   
7  4851   NaN  04851          23    ME           13     Knox County   
8  4852   NaN  04852          23    ME           15  Lincoln County   
9  4853   NaN  

In [27]:
msa_list.loc[:, 'Metropolitan statistical area'] = [x.replace('?', '-').replace(' MSA', '') for x in msa_list.loc[:, 'Metropolitan statistical area']]

cities = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Atlanta', 'Phoenix', 'Boston', 'San Francisco', 'Detroit', 'San Diego', 'Denver', 'Baltimore', 'Charlotte-', 'Orlando', 'San Antonio', 'Portland', 'Sacramento', 'Austin', 'Las Vegas', 'Cincinnati', 'Indianapolis', 'Cleveland', 'Nashville', 'Providence', 'Milwaukee', 'New Orleans', 'Hartford', 'Grand Rapids', 'Honolulu', 'Worcester', 'Greenville-', 'Sarasota', 'Boise', 'Lakeland', 'Durham', 'Spokane', 'Scranton']

zcities = [zip_msa[zip_msa['CBSA TITLE'].str.contains(city, na=False)]['CBSA TITLE'].iloc[0] for city in cities]
mcities = [msa_list[msa_list['Metropolitan statistical area'].str.contains(city, na=False)]['Metropolitan statistical area'].iloc[0] for city in cities]

for i in range(len(cities)):
    msa_list['Metropolitan statistical area'] = [x.replace(mcities[i], zcities[i]) for x in msa_list['Metropolitan statistical area']]


In [28]:
for i in range(5):
    print(zcities[i], ' | ', mcities[i])

New York-Northern New Jersey-Long Island, NY-NJ-PA  |  New York-Newark-Jersey City, NY-NJ-PA
Los Angeles-Long Beach-Santa Ana, CA  |  Los Angeles-Long Beach-Anaheim, CA
Chicago-Naperville-Joliet, IL-IN-WI  |  Chicago-Naperville-Elgin, IL-IN-WI
Houston-Sugar Land-Baytown, TX  |  Houston-The Woodlands-Sugar Land, TX
Atlanta-Sandy Springs-Marietta, GA  |  Atlanta-Sandy Springs-Alpharetta, GA


In [29]:
#print(msa_list[msa_list['Metropolitan statistical area'].str.contains('New York', na=False)]['Metropolitan statistical area'].iloc[0])
print(msa_list['Metropolitan statistical area'])

0      New York-Northern New Jersey-Long Island, NY-N...
1                   Los Angeles-Long Beach-Santa Ana, CA
2                    Chicago-Naperville-Joliet, IL-IN-WI
3                        Dallas-Fort Worth-Arlington, TX
4                         Houston-Sugar Land-Baytown, TX
                             ...                        
379                                         Danville, IL
380                                      Lewiston, ID-WA
381                                      Walla Walla, WA
382                                             Enid, OK
383                                      Carson City, NV
Name: Metropolitan statistical area, Length: 384, dtype: object


In [38]:

#print(zip_msa[zip_msa['CBSA TITLE'].str.contains('New York', na=False)].loc[:, ['CBSA TITLE'
#print(len(msa_lookup[~msa_lookup.isin(zip_msa['CBSA TITLE'])]))
#print(msa_lookup[~msa_lookup.isin(zip_msa['CBSA TITLE'])])
msa_lookup = msa_list.loc[msa_list['Rank'] <= 100, 'Metropolitan statistical area']

zip_msa1 = zip_msa[(zip_msa['CBSA TITLE'].isin(msa_lookup))]
zip_msa1 = zip_msa1.loc[:, ['ZIP5', 'COUNTY CODE', 'COUNTY NAME', 'STATE CODE', 'STATE', 'CBSA TITLE']]
zip_msa1['CBSA TITLE'] = [x.replace('--', '-') for x in zip_msa1['CBSA TITLE']]
zip_msa1['zip_num'] = zip_msa1['ZIP5']
zip_msa1['ZIP5'] = [str(i) for i in zip_msa1['ZIP5']]
zip_msa1.loc[zip_msa1['ZIP5'].str.len() == 3, 'ZIP5'] = ['00' + i for i in zip_msa1.loc[zip_msa1['ZIP5'].str.len() == 3, 'ZIP5']]
zip_msa1.loc[zip_msa1['ZIP5'].str.len() == 4, 'ZIP5'] = ['0' + i for i in zip_msa1.loc[zip_msa1['ZIP5'].str.len() == 4, 'ZIP5']]
zip_msa1['zip_code'] = zip_msa1['ZIP5']

#print(zip_msa1['CBSA TITLE'].unique())
#print(len(zip_msa1['CBSA TITLE'].unique()))
#zip_msa1['ZIP5'].unique()

zip_msa1 = zip_msa1.drop_duplicates().set_index('ZIP5')

print(zip_msa1[zip_msa1['CBSA TITLE'].str.contains('Boston')])


       COUNTY CODE       COUNTY NAME  STATE CODE STATE  \
ZIP5                                                     
01901            9      Essex County          25    MA   
01902            9      Essex County          25    MA   
01903            9      Essex County          25    MA   
01904            9      Essex County          25    MA   
01905            9      Essex County          25    MA   
...            ...               ...         ...   ...   
01464           17  Middlesex County          25    MA   
01469           17  Middlesex County          25    MA   
01471           17  Middlesex County          25    MA   
01472           17  Middlesex County          25    MA   
01474           17  Middlesex County          25    MA   

                           CBSA TITLE  zip_num zip_code  
ZIP5                                                     
01901  Boston-Cambridge-Quincy, MA-NH     1901    01901  
01902  Boston-Cambridge-Quincy, MA-NH     1902    01902  
01903  Boston

In [39]:
rdc_core = pd.read_csv("data/RDC_Inventory_Core_Metrics_Zip_History.csv", header=0, delimiter=",", encoding="utf-8", low_memory=False)

In [40]:
#print(rdc_core.head(10))
print(rdc_core.shape)
print(rdc_core['month_date_yyyymm'].min())

rdc_core.describe()

(2099207, 40)
201607


,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,median_days_on_market_mm,median_days_on_market_yy,new_listing_count,...,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,quality_flag
count,2.093776e+06,1.722876e+06,1.680143e+06,2.095350e+06,1.715901e+06,1.672116e+06,2.092807e+06,1.722004e+06,1.679063e+06,2.096596e+06,...,2.093776e+06,1.722876e+06,1.680143e+06,2.096596e+06,1.736580e+06,1.692487e+06,1.621957e+06,1.308066e+06,1.205920e+06,1.763344e+06
mean,3.391753e+05,7.485229e+00,1.027266e+01,3.462469e+01,2.456341e-02,1.792552e-03,9.117091e+01,2.141500e-01,4.217611e-01,1.650139e+01,...,4.160066e+05,7.125210e+00,9.749012e+00,5.071847e+01,2.422388e-02,4.758049e-02,7.643234e-01,1.127623e-02,1.692446e-01,5.718680e-01
std,8.216754e+05,1.371211e+03,1.718117e+03,6.139505e+01,3.563369e-01,8.014026e-01,9.564917e+01,2.066704e+00,4.301342e+00,2.709040e+01,...,9.685572e+05,1.367044e+03,1.696182e+03,8.370075e+01,3.131291e-01,7.927427e-01,9.346520e-01,6.900294e-01,1.005770e+00,4.948082e-01
min,1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,1.000000e+00,-9.995000e-01,-9.998000e-01,0.000000e+00,...,1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,-6.904550e+01,-8.232610e+01,0.000000e+00
25%,1.415000e+05,-4.170000e-02,-7.380000e-02,3.000000e+00,-1.148000e-01,-4.000000e-01,4.200000e+01,-1.280000e-01,-3.657000e-01,0.000000e+00,...,1.649290e+05,-4.140000e-02,-8.200000e-02,5.000000e+00,-9.680000e-02,-3.041000e-01,2.121000e-01,-1.111000e-01,-1.071000e-01,0.000000e+00
50%,2.390000e+05,0.000000e+00,6.800000e-02,1.300000e+01,0.000000e+00,-1.285000e-01,6.800000e+01,9.000000e-02,-7.140000e-02,4.000000e+00,...,2.750125e+05,0.000000e+00,6.500000e-02,1.800000e+01,0.000000e+00,-8.060000e-02,5.000000e-01,0.000000e+00,5.510000e-02,1.000000e+00
75%,3.874500e+05,4.910000e-02,2.597000e-01,4.100000e+01,9.380000e-02,1.429000e-01,1.100000e+02,2.966000e-01,3.151000e-01,2.200000e+01,...,4.564260e+05,4.980000e-02,2.586000e-01,6.300000e+01,8.450000e-02,1.653000e-01,1.000000e+00,1.230000e-01,3.802000e-01,1.000000e+00
max,2.790000e+08,6.989990e+05,7.989990e+05,2.647000e+03,6.800000e+01,1.030000e+02,7.370000e+03,7.505000e+02,7.595000e+02,9.440000e+02,...,2.790000e+08,6.989990e+05,7.989990e+05,2.862000e+03,6.800000e+01,1.680000e+02,9.450000e+01,7.536840e+01,9.262890e+01,1.000000e+00


In [41]:
rdc_core = rdc_core[(rdc_core['postal_code'] != ' please contact economics@realtor.com for more details.')]
rdc_core['month_date_yyyymm'] = rdc_core['month_date_yyyymm'].astype(int)
rdc_core = rdc_core[(rdc_core['month_date_yyyymm'] >= 202101) & (rdc_core['postal_code'].isin(zip_msa1.index))]

rdc_core = rdc_core.set_index('postal_code').join(zip_msa1, how='left')
rdc_core.rename(columns = {'month_date_yyyymm':'year_month', 'STATE':'State', 'CBSA TITLE':'MSA', 'COUNTY NAME': 'County'}, inplace = True)

rdc_core['year_month'] = pd.Categorical(rdc_core['year_month'], ordered=True)

rdc_core['zip_name'] = rdc_core['zip_name'].str.replace(r'\b[a-z](?=.*,)', lambda x: x.group().capitalize(), n=10, regex=True)
rdc_core['zip_name'] = rdc_core['zip_name'].str.replace(r'(?<=,\s\b)[a-z][a-z]', lambda x: x.group().upper(), n=2, regex=True)

print(rdc_core.shape)
rdc_core.columns

(291351, 46)


Index(['year_month', 'zip_name', 'median_listing_price',
       'median_listing_price_mm', 'median_listing_price_yy',
       'active_listing_count', 'active_listing_count_mm',
       'active_listing_count_yy', 'median_days_on_market',
       'median_days_on_market_mm', 'median_days_on_market_yy',
       'new_listing_count', 'new_listing_count_mm', 'new_listing_count_yy',
       'price_increased_count', 'price_increased_count_mm',
       'price_increased_count_yy', 'price_reduced_count',
       'price_reduced_count_mm', 'price_reduced_count_yy',
       'pending_listing_count', 'pending_listing_count_mm',
       'pending_listing_count_yy', 'median_listing_price_per_square_foot',
       'median_listing_price_per_square_foot_mm',
       'median_listing_price_per_square_foot_yy', 'median_square_feet',
       'median_square_feet_mm', 'median_square_feet_yy',
       'average_listing_price', 'average_listing_price_mm',
       'average_listing_price_yy', 'total_listing_count',
       'total_lis

In [42]:
rdc_hotness = pd.read_csv("data/RDC_Inventory_Hotness_Metrics_Zip_History.csv", header=0, delimiter=",", encoding="utf-8", low_memory=False)

In [43]:

print(rdc_hotness.shape)
print(rdc_hotness['month_date_yyyymm'].min())
rdc_hotness.describe
#print(rdc_hotness['postal_code'].min())

(799275, 24)
201708


<bound method NDFrame.describe of                                         month_date_yyyymm  \
0                                                  202008   
1                                                  202008   
2                                                  202003   
3                                                  201901   
4                                                  201905   
...                                                   ...   
799270                                             201801   
799271                                             201711   
799272                                             201802   
799273                                             201801   
799274  quality_flag = 1:  year-over-year figures may ...   

                                              postal_code          zip_name  \
0                                                   27565        oxford, nc   
1                                                   27514   chapel hill, nc

In [44]:
rdc_hotness = rdc_hotness[(rdc_hotness['postal_code'] != ' please contact economics@realtor.com for more details.')]
rdc_hotness['month_date_yyyymm'] = rdc_hotness['month_date_yyyymm'].astype(int)
rdc_hotness = rdc_hotness[(rdc_hotness['month_date_yyyymm'] >= 202101) & (rdc_hotness['postal_code'].isin(zip_msa1.index))]

rdc_hotness = rdc_hotness.set_index('postal_code').join(zip_msa1, how='left')
rdc_hotness.rename(columns = {'month_date_yyyymm':'year_month', 'STATE':'State', 'CBSA TITLE':'MSA', 'COUNTY NAME': 'County'}, inplace = True)

rdc_hotness['year_month'] = pd.Categorical(rdc_hotness['year_month'], categories=rdc_hotness.loc[:, 'year_month'].sort_values(ascending=True).unique(), ordered=True)

rdc_hotness['zip_name'] = rdc_hotness['zip_name'].str.replace(r'\b[a-z](?=.*,)', lambda x: x.group().capitalize(), n=10, regex=True)
rdc_hotness['zip_name'] = rdc_hotness['zip_name'].str.replace(r'(?<=,\s\b)[a-z][a-z]', lambda x: x.group().upper(), n=2, regex=True)

print(rdc_hotness.shape)
rdc_hotness.columns

C:\Users\TylerFrankenberg\AppData\Local\Temp\ipykernel_21648\1224875255.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdc_hotness['month_date_yyyymm'] = rdc_hotness['month_date_yyyymm'].astype(int)


(152706, 30)


Index(['year_month', 'zip_name', 'nielsen_hh_rank', 'hotness_rank',
       'hotness_rank_mm', 'hotness_rank_yy', 'hotness_score', 'supply_score',
       'demand_score', 'median_days_on_market', 'median_days_on_market_mm',
       'median_dom_mm_day', 'median_days_on_market_yy', 'median_dom_yy_day',
       'median_dom_vs_us', 'ldp_unique_viewers_per_property_mm',
       'ldp_unique_viewers_per_property_yy',
       'ldp_unique_viewers_per_property_vs_us', 'median_listing_price',
       'median_listing_price_mm', 'median_listing_price_yy',
       'median_listing_price_vs_us', 'quality_flag', 'COUNTY CODE', 'County',
       'STATE CODE', 'State', 'MSA', 'zip_num', 'zip_code'],
      dtype='object')

In [45]:
zip_msa1.to_csv('data/zip_msa1.csv', index=False, encoding='latin1')
rdc_hotness.to_csv('data/rdchotness.csv', index=False, encoding='utf-8')
rdc_core.to_csv('data/rdc_core.csv', index=False, encoding='utf-8')